<a href="https://colab.research.google.com/github/DuplamenteH/Machine-Learning/blob/master/Clustering_Basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # *Estudo de alguns algoritmos de clustering ex: K-means, DBSCAN, mean shift*

# Aula 1

instalação

In [ ]:
pip install plotly==4.1.1


In [ ]:
pip install biokit

### Analise dos dados e importações de libs.

importações

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from biokit.viz import corrplot
import matplotlib.pyplot as plt



In [ ]:
path_arq = 'https://gist.githubusercontent.com/tijptjik/9408623/raw/b237fa5848349a14a14e5d4107dc7897c21951f5/wine.csv'

In [ ]:
df = pd.read_csv(path_arq, sep=',')

In [ ]:
print("Dimensões de Dataset {}, {}".format(df.shape[0], df.shape[1]))
df.head(10)

renomeando as features

In [ ]:
colunas = {
    'Alcohol':'Alcool',
    'Ash' : 'PO',
    'Acl':'Alcalinidade_po',
    'Mg' : 'Magnesio',
    'Color.int' : 'Intensidade_cor'
}
df = df.rename(columns=colunas)
df.head()

In [ ]:
df.describe()

### Insights:

* Vemos que a base de dados não possui dados faltantes.
* Média(e STD) bem variada ao longo do DF.-> Processo de adquação
* Variação bem alta do proline.

### vendo as relações

In [ ]:
matriz_corr = df.corr()

In [ ]:
corr_graf = corrplot.Corrplot(matriz_corr)
corr_graf.plot(upper = 'ellipse', fontsize = 'x-large')
fig = plt.gcf()
fig.set_size_inches(14,14)
fig.show()